## Necessary Functions
This notebook contains the key libraries and functions needed to parse the PalmSens EIS data, graph it and fit it to an equivalent circuit

In [5]:
#Import Libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import impedance
from impedance import preprocessing
from impedance.models.circuits import CustomCircuit
from impedance.visualization import plot_nyquist
import math

In [5]:
def format_EIS(file_list,directory):
    """
    This function takes a list of csv files of Palmsens EIS data and saves the data as
    a dictionary of dataframes
    
    file_list: list of strings of file names
    directory: string of location of files on computer
    
    return: dictionary with key = file name and value = data
    """
    
    EIS_data = {}
    print("File count:")
    
    #Loop through EIS files 
    for x in range(len(file_list)):
        print("file",x+1)
        with open(os.path.join(directory, file_list[x]), 'rb') as file:
            content = file.read().decode('utf-16')
        
        # Split the csv into different parts
        sections = content.split('\r\n')
        sections.pop()
        sections.pop()
        
        loc_of_files = []
        units_start = []
        data_start = []
        spacer = []
        titles = []
        n_data=1
        

        # Identify where data labels and data points are
        for i, section in enumerate(sections):
            if section.find("Measurement") == 0:
                loc_of_files.append(i)
                units_start.append(i+4)
                data_start.append(i+5)
            if section == "":
                spacer.append(i)
        spacer.append(len(sections))
        
        
        for number in range(len(loc_of_files)):
            data_points = []
            run_titles = []

            # Create Column labels of Units in file
            units = sections[units_start[number]].split(',')
            cols = []
            count_t = 1
            count_f = 1
            count_p = 1
            count_curr = 1
            count_Z1 = 1
            count_Z2 = 1
            count_Z3 = 1
            count_Z4 = 1
            count_Cs = 1
            count_Edc = 1
            count_Eac = 1
            for col in units:
                if col == 'freq / Hz':
                    cols.append(f'freq_{count_f}')
                    count_f += 1
                    continue
                if col == 'neg. Phase / °':
                    cols.append(f'phase_{count_p}')
                    count_p += 1
                    continue
                if col == 'Idc / uA':
                    cols.append(f'current_{count_curr}')
                    count_curr += 1
                    continue
                if col == 'Z / Ohm':
                    cols.append(f'|Z|_{count_Z1}')
                    count_Z1 += 1
                if col == "Z' / Ohm":
                    cols.append(f'Zr_{count_Z2}')
                    count_Z2 += 1
                if col == "-Z'' / Ohm":
                    cols.append(f'-Zi_{count_Z3}')
                    count_Z3 += 1
                if col == "Z'' / Ohm":
                    cols.append(f'Zi_{count_Z3}')
                    count_Z4 += 1
                if col == "Cs / F":
                    cols.append(f'cs_{count_Cs}')
                    count_Cs += 1
                if col == "Edc / V":
                    cols.append(f'Edc_{count_Edc}')
                    count_Edc += 1
                if col == "Eac / V":
                    cols.append(f'Eac_{count_Eac}')
                    count_Eac += 1   
                if col == "Time / s":
                    cols.append(f'Time_{count_t}')
                    count_t += 1
            # Parse data        
            for i in range(data_start[number],spacer[number]-1):
                section = sections[i]
                data_points.append(section.strip(',').split(','))
            
                
            data_points = [[float(value)for value in row] for row in data_points]
            
            # Create the dataframe with units
            df = pd.DataFrame(data_points, columns=cols)
            
            # Assign name to dataframe and add to dictionary
            
            data_title = sections[loc_of_files[number]].split(",")[1]
            if data_title == "Impedance Spectroscopy":
                title = file_list[x].split('.')[0]
                EIS_data[f'{title}_{n_data}'] = df
                print(f"   >{title}_{n_data}")
                n_data += 1
            else:
                title = data_title
                titles.append(data_title)
                n = titles.count(data_title)
                EIS_data[f'{title}_{n}'] = df
                print(f"   >{title}_{n}")

    return EIS_data

In [6]:
def visualize_EIS(EIS_dictionary):
    """
    This function takes a dictionary of EIS data files and returns a graph of the 
    impedance and phase versus frequency
    
    EIS_dictionary: dictionary with key = file name and value = data
        looks for data titles from "format_files" function
    
    return: plot of impedance/phase versus frequency
    """
    #Creat plot with two axes
    with plt.style.context('classic'):
        plt.ioff()
        fig = plt.figure( figsize = (10,6) )
        fig.patch.set_facecolor('white')
        ax = fig.add_subplot(1,1,1)
        ax2 = ax.twinx()

        #create color palette
        palette = sns.color_palette("Paired",2*len(list(EIS_dictionary.keys())))
        

        for i, dataset in enumerate(list(EIS_dictionary.keys())):
            ax.plot(EIS_dictionary[dataset]['freq_1'], EIS_dictionary[dataset]['|Z|_1'], color = palette[2*i+1],label = dataset, linewidth = '2')
            ax2.plot(EIS_dictionary[dataset]['freq_1'], EIS_dictionary[dataset]['phase_1'], color = palette[2*i],label = dataset, linewidth = '2', linestyle = "--")

        ax.legend(bbox_to_anchor=(1.02, 1), loc = 'upper left')

        # Now we can label the axes. 
        ax.set_xlabel('$ Frequency (Hz) $', fontsize = 20)
        ax.set_ylabel('$ |Z| (Ω) $', fontsize = 20)
        ax2.set_ylabel('$ Phase (°) $', fontsize = 20)

        ax.set_yscale("log")
        ax.set_xscale("log")

        # adjust the length and tickness of tick marks
        ax.tick_params(width = 1.5, length = 6, axis = 'y')
        ax.tick_params(width = 1.5, length = 6, axis = 'x')
        ax2.tick_params(width = 1.5, length = 6, axis = 'y')
        ax2.tick_params(width = 1.5, length = 6, axis = 'x')
    
    display(fig)
    return

In [7]:
def fit_EIS(EIS_dictionary,fits=False):
    """
    This function takes a dictionary of EIS data files and returns 
    fitted values assuming a debye circuit.
    Uses python package "impedance", may need to install
    
    EIS_dictionary: dictionary with key = file name and value = data
    looks for data titles from "format_files" function
    fits = boolean if the code will graph the fits, default to False
    
    return: panda dataframe of fitted values
    """
    import math
    # Create dataframe to put fitted values
    df_EIS = pd.DataFrame()
    df_EIS['Dataset'] = ['R electronic (Ω)',"R ionic (Ω)","Q", "alpha"]

    n = len(list(EIS_dictionary.keys()))
    
    plt.ioff()
    fig = plt.figure(figsize = (6,3*n))
    fig.patch.set_facecolor('white')
        
    #Fit each dataset
    for i, dataset in enumerate(list(EIS_dictionary.keys())):
        n_values = len(EIS_dictionary[dataset]['|Z|_1'])
        
        Rhigh = EIS_dictionary[dataset]['|Z|_1'][1]
        Rlow = EIS_dictionary[dataset]['|Z|_1'][n_values-1]
        Fhigh = EIS_dictionary[dataset]['freq_1'][1]
        Flow = EIS_dictionary[dataset]['freq_1'][n_values-1]
        C = 1/(Rlow/Flow)

        # set initial guesses and Debye circuit
        # can change here, may come back and add option to set this in function
        initial_guess=[Rlow, Rhigh, C, 0.9]
        circ_string='p(R1,R2-CPE1)'

        # Create datasets for frequency and impedance in complex form
        f = EIS_dictionary[dataset]['freq_1']
        Z = []
        for t in range(len(EIS_dictionary[dataset]['|Z|_1'])):
            Z.append(complex(EIS_dictionary[dataset]['|Z|_1'][t]*math.cos(math.radians(EIS_dictionary[dataset]['phase_1'][t]*-1)),EIS_dictionary[dataset]['|Z|_1'][t]*math.sin(math.radians(EIS_dictionary[dataset]['phase_1'][t]*-1))))
       
        #Fit and save parameters
        circuit = CustomCircuit(circ_string, initial_guess=initial_guess, name=dataset)
        circuit.fit(f, Z, weight_by_modulus=True)
        df_EIS[dataset] = circuit.parameters_
        Z_pred = circuit.predict(f)

        #create color palette
        palette = sns.color_palette("Paired",2*len(list(EIS_dictionary.keys())))
        
        ax = fig.add_subplot(n,1,i+1)
        
        ax.plot(EIS_dictionary[dataset]['freq_1'], EIS_dictionary[dataset]['|Z|_1'], color = palette[2*i+1],label = dataset, linewidth = '2')
        ax.plot(EIS_dictionary[dataset]['freq_1'], abs(Z_pred), color = palette[2*i],label = dataset+" fit", linewidth = '2', linestyle = "dotted")
        ax.set_yscale("log")
        ax.set_xscale("log")
        ax.legend(bbox_to_anchor=(1.02, 1), loc = 'upper left')
        ax.set_xlabel('$ Frequency (Hz) $', fontsize = 10)
        ax.set_ylabel('$ |Z| (Ω) $', fontsize = 10) 
    
    df_EIS = df_EIS.T
    df_EIS.columns = df_EIS.iloc[0]
    df_EIS = df_EIS[1:]
    df_EIS['Effective Capacitance (uF)'] = (df_EIS['R electronic (Ω)']*df_EIS['Q'])**(1/df_EIS['alpha'])*1000000/(df_EIS['R electronic (Ω)'])
    if fits:
        display(fig)

    return df_EIS